# Clustering Crypto

In [1]:
!pip install -U altair

Requirement already up-to-date: altair in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (4.1.0)
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [2]:
import altair as alt

# Amazon SageMaker 
import sagemaker


In [3]:
# Initial imports
import requests
import pandas as pd
from path import Path
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Fetching Cryptocurrency Data

In [4]:
# Alternatively, use the provided csv file:
file_path = Path("Resources/crypto_data.csv")

# Create a DataFrame
crypto_df = pd.read_csv(file_path)
crypto_df.head()

Unnamed: 0   CoinName Algorithm  IsTrading ProofType  TotalCoinsMined  \
0         42    42 Coin    Scrypt       True   PoW/PoS     4.199995e+01   
1        365    365Coin       X11       True   PoW/PoS              NaN   
2        404    404Coin    Scrypt       True   PoW/PoS     1.055185e+09   
3        611  SixEleven   SHA-256       True       PoW              NaN   
4        808        808   SHA-256       True   PoW/PoS     0.000000e+00   

  TotalCoinSupply  
0              42  
1      2300000000  
2       532000000  
3          611000  
4               0

### Data Preprocessing

In [5]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df = crypto_df.set_index(crypto_df.columns[0])
crypto_df.head()

CoinName Algorithm  IsTrading ProofType  TotalCoinsMined  \
Unnamed: 0                                                              
42            42 Coin    Scrypt       True   PoW/PoS     4.199995e+01   
365           365Coin       X11       True   PoW/PoS              NaN   
404           404Coin    Scrypt       True   PoW/PoS     1.055185e+09   
611         SixEleven   SHA-256       True       PoW              NaN   
808               808   SHA-256       True   PoW/PoS     0.000000e+00   

           TotalCoinSupply  
Unnamed: 0                  
42                      42  
365             2300000000  
404              532000000  
611                 611000  
808                      0

In [6]:
# Keep only cryptocurrencies that are trading
crypto_df = crypto_df.loc[
    crypto_df ["IsTrading"]== True
]

crypto_df.head()

CoinName Algorithm  IsTrading ProofType  TotalCoinsMined  \
Unnamed: 0                                                              
42            42 Coin    Scrypt       True   PoW/PoS     4.199995e+01   
365           365Coin       X11       True   PoW/PoS              NaN   
404           404Coin    Scrypt       True   PoW/PoS     1.055185e+09   
611         SixEleven   SHA-256       True       PoW              NaN   
808               808   SHA-256       True   PoW/PoS     0.000000e+00   

           TotalCoinSupply  
Unnamed: 0                  
42                      42  
365             2300000000  
404              532000000  
611                 611000  
808                      0

In [7]:
# Keep only cryptocurrencies with a working algorithm
#Check for nulls
crypto_df.isnull().sum()

#All cryptocurrencies have a working algorithm

CoinName             0
Algorithm            0
IsTrading            0
ProofType            0
TotalCoinsMined    459
TotalCoinSupply      0
dtype: int64

In [8]:
# Remove the 'IsTrading' column
crypto_df.drop(columns=['IsTrading'], inplace = True)
crypto_df.head()

CoinName Algorithm ProofType  TotalCoinsMined TotalCoinSupply
Unnamed: 0                                                                
42            42 Coin    Scrypt   PoW/PoS     4.199995e+01              42
365           365Coin       X11   PoW/PoS              NaN      2300000000
404           404Coin    Scrypt   PoW/PoS     1.055185e+09       532000000
611         SixEleven   SHA-256       PoW              NaN          611000
808               808   SHA-256   PoW/PoS     0.000000e+00               0

In [9]:
# Remove rows with at least 1 null value

# Drop nulls
crypto_df = crypto_df.dropna().copy()
crypto_df.head()

#crypto_df.isnull().sum()

CoinName Algorithm ProofType  TotalCoinsMined TotalCoinSupply
Unnamed: 0                                                                
42            42 Coin    Scrypt   PoW/PoS     4.199995e+01              42
404           404Coin    Scrypt   PoW/PoS     1.055185e+09       532000000
808               808   SHA-256   PoW/PoS     0.000000e+00               0
1337        EliteCoin       X13   PoW/PoS     2.927942e+10    314159265359
BTC           Bitcoin   SHA-256       PoW     1.792718e+07        21000000

In [10]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df = crypto_df.loc[
    crypto_df ["TotalCoinsMined"] > 0

]

crypto_df.head()

CoinName Algorithm ProofType  TotalCoinsMined TotalCoinSupply
Unnamed: 0                                                                
42            42 Coin    Scrypt   PoW/PoS     4.199995e+01              42
404           404Coin    Scrypt   PoW/PoS     1.055185e+09       532000000
1337        EliteCoin       X13   PoW/PoS     2.927942e+10    314159265359
BTC           Bitcoin   SHA-256       PoW     1.792718e+07        21000000
ETH          Ethereum    Ethash       PoW     1.076842e+08               0

In [11]:
# Drop rows where there are 'N/A' text values
crypto_df = crypto_df[
    crypto_df.iloc [ : , :] != "N/A"
]

crypto_df.head()

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


CoinName Algorithm ProofType  TotalCoinsMined TotalCoinSupply
Unnamed: 0                                                                
42            42 Coin    Scrypt   PoW/PoS     4.199995e+01              42
404           404Coin    Scrypt   PoW/PoS     1.055185e+09       532000000
1337        EliteCoin       X13   PoW/PoS     2.927942e+10    314159265359
BTC           Bitcoin   SHA-256       PoW     1.792718e+07        21000000
ETH          Ethereum    Ethash       PoW     1.076842e+08               0

In [12]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df 
coins_name_df = crypto_df[["CoinName"]]
coins_name_df.head()

CoinName
Unnamed: 0           
42            42 Coin
404           404Coin
1337        EliteCoin
BTC           Bitcoin
ETH          Ethereum

In [13]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df = crypto_df.drop(columns=['CoinName'])
crypto_df.head()

Algorithm ProofType  TotalCoinsMined TotalCoinSupply
Unnamed: 0                                                     
42            Scrypt   PoW/PoS     4.199995e+01              42
404           Scrypt   PoW/PoS     1.055185e+09       532000000
1337             X13   PoW/PoS     2.927942e+10    314159265359
BTC          SHA-256       PoW     1.792718e+07        21000000
ETH           Ethash       PoW     1.076842e+08               0

In [14]:
# Create dummy variables for text features
crypto_df_encoded = pd.get_dummies(crypto_df, columns=["Algorithm", "ProofType"])
crypto_df_encoded_copy =crypto_df_encoded.copy()


crypto_df_encoded.head()


TotalCoinsMined TotalCoinSupply  Algorithm_1GB AES Pattern Search  \
Unnamed: 0                                                                      
42             4.199995e+01              42                                 0   
404            1.055185e+09       532000000                                 0   
1337           2.927942e+10    314159265359                                 0   
BTC            1.792718e+07        21000000                                 0   
ETH            1.076842e+08               0                                 0   

            Algorithm_536  Algorithm_Argon2d  Algorithm_BLAKE256  \
Unnamed: 0                                                         
42                      0                  0                   0   
404                     0                  0                   0   
1337                    0                  0                   0   
BTC                     0                  0                   0   
ETH                     0                  0                   0   

            Algorithm_Blake  Algorithm_Blake2S  Algorithm_Blake2b  \
Unnamed: 0                                                          
42                        0                  0                  0   
404                       0                  0                  0   
1337                      0                  0                  0   
BTC                       0                  0                  0   
ETH                       0                  0                  0   

            Algorithm_C11  ...  ProofType_PoW/PoS  ProofType_PoW/PoS   \
Unnamed: 0                 ...                                          
42                      0  ...                  1                   0   
404                     0  ...                  1                   0   
1337                    0  ...                  1                   0   
BTC                     0  ...                  0                   0   
ETH                     0  ...                  0                   0   

            ProofType_PoW/PoW  ProofType_PoW/nPoS  ProofType_Pos  \
Unnamed: 0                                                         
42                          0                   0              0   
404                         0                   0              0   
1337                        0                   0              0   
BTC                         0                   0              0   
ETH                         0                   0              0   

            ProofType_Proof of Authority  ProofType_Proof of Trust  \
Unnamed: 0                                                           
42                                     0                         0   
404                                    0                         0   
1337                                   0                         0   
BTC                                    0                         0   
ETH                                    0                         0   

            ProofType_TPoS  ProofType_Zero-Knowledge Proof  ProofType_dPoW/PoW  
Unnamed: 0                                                                      
42                       0                               0                   0  
404                      0                               0                   0  
1337                     0                               0                   0  
BTC                      0                               0                   0  
ETH                      0                               0                   0  

[5 rows x 98 columns]

In [15]:
# Standardize data
crypto_scaled = StandardScaler().fit_transform(crypto_df_encoded_copy)
print(crypto_scaled)


[[-0.11710817 -0.1528703  -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.09396955 -0.145009   -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [ 0.52494561  4.48942416 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 ...
 [-0.09561336 -0.13217937 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.11694817 -0.15255998 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.11710536 -0.15285552 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]]


### Reducing Dimensions Using PCA

In [16]:
# Use PCA to reduce dimensions to 3 principal components

# Initialize PCA model
pca = PCA(n_components=3)


# Get 3 principal components for the  data.
pcs = pca.fit_transform(crypto_scaled )



In [17]:
# Create a DataFrame with the principal components data
pca_columns = ["PC 1", "PC 2" ,"PC 3"]
pcs_df= pd.DataFrame(
    data=pcs,
    columns=pca_columns,
    index=crypto_df_encoded.index
)

pcs_df.head(10)


PC 1      PC 2      PC 3
Unnamed: 0                              
42         -0.335172  1.042956 -0.494827
404        -0.318525  1.043025 -0.495168
1337        2.319105  1.622003 -0.530247
BTC        -0.153693 -1.359293  0.188360
ETH        -0.164267 -2.079738  0.326104
LTC        -0.165083 -1.095418 -0.024932
DASH       -0.400754  1.172968 -0.401307
XMR        -0.164388 -2.238472  0.436820
ETC        -0.162711 -2.079842  0.326091
ZEC        -0.115678 -2.038674  0.232144

### Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [18]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)



# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

source=df_elbow

alt.Chart(source).mark_line().encode(
    x="k",
    y="inertia"
)


alt.Chart(...)

Running K-Means with `k=4

In [19]:
# Initialize the K-Means model
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)

# Create a new DataFrame including predicted clusters and cryptocurrencies features
pcs_df["class"] = model.labels_


#concat all 3 dataframes
clustered_df = pd.concat(([crypto_df, pcs_df,coins_name_df]),axis=1)


#Reorder columns
clustered_df = clustered_df[["Algorithm", "ProofType", "TotalCoinsMined",
            "TotalCoinSupply", "PC 1", "PC 2", "PC 3", "CoinName", "class"]]

clustered_df.head(10)

Algorithm ProofType  TotalCoinsMined TotalCoinSupply  \
Unnamed: 0                                                              
42                  Scrypt   PoW/PoS     4.199995e+01              42   
404                 Scrypt   PoW/PoS     1.055185e+09       532000000   
1337                   X13   PoW/PoS     2.927942e+10    314159265359   
BTC                SHA-256       PoW     1.792718e+07        21000000   
ETH                 Ethash       PoW     1.076842e+08               0   
LTC                 Scrypt       PoW     6.303924e+07        84000000   
DASH                   X11   PoW/PoS     9.031294e+06        22000000   
XMR         CryptoNight-V7       PoW     1.720114e+07               0   
ETC                 Ethash       PoW     1.133597e+08       210000000   
ZEC               Equihash       PoW     7.383056e+06        21000000   

                PC 1      PC 2      PC 3          CoinName  class  
Unnamed: 0                                                         
42         -0.335172  1.042956 -0.494827           42 Coin      0  
404        -0.318525  1.043025 -0.495168           404Coin      0  
1337        2.319105  1.622003 -0.530247         EliteCoin      0  
BTC        -0.153693 -1.359293  0.188360           Bitcoin      1  
ETH        -0.164267 -2.079738  0.326104          Ethereum      1  
LTC        -0.165083 -1.095418 -0.024932          Litecoin      1  
DASH       -0.400754  1.172968 -0.401307              Dash      0  
XMR        -0.164388 -2.238472  0.436820            Monero      1  
ETC        -0.162711 -2.079842  0.326091  Ethereum Classic      1  
ZEC        -0.115678 -2.038674  0.232144             ZCash      1

### Visualizing Results

#### 3D-Scatter with Clusters

In [20]:
# Create a 2D-Scatter with the PCA data and the clusters
source = clustered_df

alt.Chart(source).mark_circle(size=60).encode(
    x="PC 1:Q",
    y="PC 2:Q",
    color='class:N',
    tooltip=['CoinName', 'Algorithm', 'TotalCoinsMined', 'TotalCoinSupply']
    
).interactive()

alt.Chart(...)

#### Table of Tradable Cryptocurrencies

In [21]:
# Table with tradable cryptos
#Select required columns
clustered_df = clustered_df[["CoinName", "Algorithm", "ProofType", 
                             "TotalCoinSupply", "TotalCoinsMined", "class"]]

pd.set_option("display.max_rows", None,)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth",None)

clustered_df

CoinName               Algorithm  \
Unnamed: 0                                                             
42                                   42 Coin                  Scrypt   
404                                  404Coin                  Scrypt   
1337                               EliteCoin                     X13   
BTC                                  Bitcoin                 SHA-256   
ETH                                 Ethereum                  Ethash   
LTC                                 Litecoin                  Scrypt   
DASH                                    Dash                     X11   
XMR                                   Monero          CryptoNight-V7   
ETC                         Ethereum Classic                  Ethash   
ZEC                                    ZCash                Equihash   
BTS                                Bitshares                 SHA-512   
DGB                                 DigiByte                Multiple   
BTCD                             BitcoinDark                 SHA-256   
XPY                                  PayCoin                 SHA-256   
PRC                              ProsperCoin                  Scrypt   
KOBO                                KoboCoin                     X15   
SPR                               Spreadcoin                     X11   
ARG                                 Argentum                  Scrypt   
AUR                              Aurora Coin                  Scrypt   
BLU                                 BlueCoin                  Scrypt   
XMY                               MyriadCoin                Multiple   
MOON                                MoonCoin                  Scrypt   
ZET                                 ZetaCoin                 SHA-256   
SXC                                  SexCoin                  Scrypt   
QTL                                  Quatloo                  Scrypt   
ENRG                              EnergyCoin                  Scrypt   
QRK                                QuarkCoin                   Quark   
RIC                                  Riecoin                 Groestl   
DGC                             Digitalcoin                   Scrypt   
BTB                                   BitBar                  Scrypt   
CAT                                  Catcoin                  Scrypt   
CBX                            CryptoBullion                  Scrypt   
CCN                                CannaCoin                  Scrypt   
CRYPT                              CryptCoin                     X11   
CSC                               CasinoCoin                  Scrypt   
DMD                                  Diamond                 Groestl   
XVG                                    Verge                Multiple   
DVC                                  DevCoin                 SHA-256   
EAC                                EarthCoin                  Scrypt   
EFL                                 E-Gulden                  Scrypt   
EMC2                             Einsteinium                  Scrypt   
EMD                                  Emerald                  Scrypt   
EXCL                          Exclusive Coin                     PoS   
FLT                              FlutterCoin                  Scrypt   
FRK                                   Franko                  Scrypt   
FTC                              FeatherCoin               NeoScrypt   
GDC                                GrandCoin                  Scrypt   
GLC                               GlobalCoin                  Scrypt   
GLD                                 GoldCoin                  Scrypt   
HBN                              HoboNickels                  Scrypt   
HYP                               HyperStake                     X11   
IFC                            Infinite Coin                  Scrypt   
IOC                                   IOCoin                     X11   
IXC                                   IXcoin                 SHA-256   
KGC 

In [22]:
# Print the total number of tradable cryptocurrencies

print(f"The total number of tradable cryptocurrencies is:{len(clustered_df)}")

The total number of tradable cryptocurrencies is:532


#### Scatter Plot with Tradable Cryptocurrencies

In [23]:
# Scale data to create the scatter plot
columns = [ "TotalCoinSupply", "TotalCoinsMined"]
clustered_scaled = StandardScaler().fit_transform(clustered_df[columns])
print(clustered_scaled[0:5])

[[-0.1528703  -0.11710817]
 [-0.145009   -0.09396955]
 [ 4.48942416  0.52494561]
 [-0.15255998 -0.11671506]
 [-0.1528703  -0.11474682]]


In [24]:
#create a dataframe

columns = ["TotalCoinSupply", "TotalCoinsMined"]
scaled_df = pd.DataFrame(
    data=clustered_scaled,
    index=clustered_df.index
    ,columns=columns)


In [25]:
#include additional columns- "class" and "CoinName"
scaled_df[["CoinName", "class"]]=clustered_df[["CoinName", "class"]]
scaled_df.head()

TotalCoinSupply  TotalCoinsMined   CoinName  class
Unnamed: 0                                                    
42                -0.152870        -0.117108    42 Coin      0
404               -0.145009        -0.093970    404Coin      0
1337               4.489424         0.524946  EliteCoin      0
BTC               -0.152560        -0.116715    Bitcoin      1
ETH               -0.152870        -0.114747   Ethereum      1

In [26]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
source = scaled_df

alt.Chart(source).mark_circle(size=60).encode(
    x="TotalCoinsMined:Q",
    y="TotalCoinSupply:Q",
    color='class:N',
    tooltip=['CoinName']
    
    
).interactive()

alt.Chart(...)

In [30]:
# Delete Amazon SageMaker 
